发布订阅模式又叫观察者模式，它定义对象间的一种一对多的依赖关系，当一个对象的状态发生改变时，所有依赖于它的对象都将得到通知。

在javascript 中，我们一般用事件模型来替代传统的发布订阅模式。

发布订阅实现
- 指定发布者
- 发布者添加缓存列表，存放回调函数以便通知订阅者
- 发布消息时，发布者遍历缓存列表，依次触发存放的订阅者回调函数

In [1]:
(function(){
    var salesOffices = {};
    salesOffices.clientList = [];
    
    salesOffices.listen = function(fn){
        this.clientList.push(fn)
    }
    
    salesOffices.trigger = function(){
        for(var i =0,fn;fn = this.clientList[i++];){
            fn.apply(this,arguments);
        }
    }
    
    salesOffices.listen(function(price,squareMeter){
        console.log(`price is ${price}`)
        console.log(`squareMeter is ${squareMeter}`)
    })
    
    salesOffices.listen(function(price,squareMeter){
        console.log(`price is ${price}`)
        console.log(`squareMeter is ${squareMeter}`)
    })
    
    salesOffices.trigger(20000,88)
    salesOffices.trigger(300000,110)
})()

price is 20000
squareMeter is 88
price is 20000
squareMeter is 88
price is 300000
squareMeter is 110
price is 300000
squareMeter is 110


In [2]:
(function(){
    var salesOffices = {};
    salesOffices.clientList = {};
    
    salesOffices.listen = function(key,fn){
        if(!this.clientList[key]){
            this.clientList[key] = []
        }
        this.clientList[key].push(fn);
    }
    
    salesOffices.trigger = function(){
        var key = Array.prototype.shift.call(arguments),
            fns = this.clientList[key];
        
        if(!fns||fns.length === 0){
            return false;
        }
        
        for(var i = 0,fn;fn = fns[i++];){
            fn.apply(this, arguments);
        }
    }
    
    salesOffices.listen('squareMeter88',function(price){
        console.log(`price is ${price}`)
    })
    
    salesOffices.listen('squareMeter110',function(price){
        console.log(`price is ${price}`)
    })
    
    salesOffices.trigger('squareMeter88',20000)
    salesOffices.trigger('squareMeter110',300000)
})()

price is 20000
price is 300000


In [5]:
(function(){
    var event = {
        clientList:[],
        listen:function(key,fn){
            if(!this.clientList[key]){
                this.clientList[key]=[];
            }
            this.clientList[key].push(fn)
        },
        trigger:function(){
            var key = Array.prototype.shift.call(arguments),
                fns = this.clientList[key];
            
            if(!fns || fns.length ===0){
                return false;
            }
            
            for(var i =0,fn;fn = fns[i++];){
                fn.apply(this,arguments);
            }
        },
        remove:function(key,fn){
            var fns = this.clientList[key];
            if(!fns){
                return false;
            }
            if(!fn){
                fns&&(fns.length = 0)
            }else {
                for(var l = fns.length-1;l>=0;l--){
                    var _fn = fns[l];
                    if(_fn === fn){
                        fns.splice(1,1);
                    }
                }
            }
        }
    }
    
    var installEvent = function(obj){
        for(var i in event){
            obj[i] = event[i];
        }
    }
    
    //exec
    
    var salesOffices = {};
    var fn1,fn2;
    installEvent(salesOffices);
    salesOffices.listen('squareMeter88',fn1 = function(price){
        console.log(`price is ${price}`)
    })
    
    salesOffices.listen('squareMeter88',fn2 = function(price){
        console.log(`price is ${price}`)
    })
    
    salesOffices.listen('squareMeter110',function(price){
        console.log(`price is ${price}`)
    })
    
    
    salesOffices.trigger('squareMeter88',2000)
    salesOffices.remove('squareMeter88',fn1)
    salesOffices.trigger('squareMeter88',2000)
    salesOffices.trigger('squareMeter110',3000)
})()

price is 2000
price is 2000
price is 2000
price is 3000


In [7]:
(function(){
    var Event = (function(){
        var clientList = {},
            listen,
            trigger,
            remove;
        
        listen = function(key, fn){
            if(!clientList[key]){
                clientList[key] = [];
            }
            clientList[key].push(fn);
        }
        
        trigger = function(){
            var key = Array.prototype.shift.call(arguments),
                fns = clientList[key];
            if(!fns ||fns.length === 0){
                return false;
            }
            
            for(var i =0,fn;fn = fns[i++];){
                fn.apply(this,arguments);
            }
        }
        
        remove = function(key,fn){
            var fns = clientList[key];
            if(!fns){
                return false;
            }
            if(!fn){
                fns && (fns.length = 0)
            }else{
                for(var l = fns.length-1;l>=0;l--){
                    var _fn = fns[l];
                    if(_fn === fn){
                        fns.splice(1,1)
                    }
                }
            }
        }
        
        return {
            listen:listen,
            trigger:trigger,
            remove:remove
        }
    })();
    
    Event.listen('squareMeter88',function(price){
        console.log(`price is ${price}`)
    })
    
    Event.trigger('squareMeter88',20000)
})()

price is 20000


In [ ]:
(function(){
    var Event = (function(){
        var golbal = this,
            Event,
            _default = 'default';
        
        Event = function(){
            var _listen,
                _trigger,
                _remove,
                _slice = Array.prototype.slice,
                _shift = Array.prototype.shift,
                _unshift = Array.prototype.unshift,
                namespaceCache = {},
                _create,
                find,
                each = function(ary,fn){
                    var ret ;
                    for(var i =0,l = ary.length;i<l;i++){
                        var n = ary[i];
                        ret = fn.call(n,i,n);
                    }
                    return ret;
                }
            
            _listen = function(key,fn,cache){
                if(!cache[key]){
                    cache[key] = [];
                }
                cache[key].push(fn)
            }
            
            _remove = function(key,cache,fn){
                if(cache[key]){
                    if(fn){
                        for(var i = cache[key].length;i>=0;i--){
                            if(cache[key][i]===fn){
                                cache[key.splice(i,1)]
                            }
                        }
                    }else {
                        cache[key] = []
                    }
                }
            }
            
            _trigger = function(){
                var cache = _shift.call(arguments),
                    key = _shift.call(arguments),
                    args = arguments,
                    _self = this,
                    ret,
                    stack = cache[key];
                
                if(!stack ||!stack.length){
                    return;
                }
                
                return each(stack,function(){
                    return this.apply(_self,args)
                })
            }
            
            _create = function(){
                var namespace = namespace || _default;
                var cache = {},
                    offlineStack =[],
                    ret = {
                        listen:function(key,fn,last){
                            _listen(key,fn,cache);
                            if(offlineStack === null){
                                return;
                            }
                            if(last ==='last'){
                                offlineStack.length && offlineStack.pop()();
                            }else{
                                each(offlineStack,function(){
                                    this();
                                })
                            }
                            offlineStack = null;
                        },
                        one:function(key,fn,last){
                            _remove(key,cache);
                            this.listen(key,fn,last)
                        },
                        remove:function(key,fn){
                            _remove(key,cache,fn);
                        },
                        trigger:function(){
                            var fn,
                                args,
                                _self = this;
                            _unshift.call(arguments,cache);
                            args = arguments;
                            fn = function(){
                                return _trigger.apply(_self,args);
                            }
                            if(offlineStack){
                                return offlineStack.push(fn)
                            }
                            return fn()
                        }
                    }
                return namespace ?
                    (namespaceCache[namespace]?namespaceCache[namespace]:namespaceCache[namespace] = ret):ret
            }
            
            return {
                create:_create,
                one:function(key,fn,last){
                    var event = this.create();
                    event.one(key,fn,last)
                },
                remove:function(key,fn){
                    var event = this.create();
                    event.remove(key,fn);
                },
                listen:function(key,fn,last){
                    var event = this.create();
                    event.listen(key,fn,last)
                },
                trigger:function(){
                    var event = this.create();
                    event.trigger.apply(this,arguments)
                }
            }
        }()
        
        return Event;
    })()
})()